<a href="https://colab.research.google.com/github/RamsteinWR/your-first-kaggle-submission/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/agconti/kaggle-titanic.git

Cloning into 'kaggle-titanic'...
remote: Enumerating objects: 472, done.
remote: Total 472 (delta 0), reused 0 (delta 0), pack-reused 472
Receiving objects: 100% (472/472), 7.61 MiB | 25.82 MiB/s, done.
Resolving deltas: 100% (216/216), done.


SyntaxError: ignored

In [4]:
!pwd # path of current directory

/content


In [6]:
cd sample_data


[Errno 2] No such file or directory: 'sample_data'
/content/sample_data


In [7]:
!ls

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [8]:
! git clone https://github.com/agconti/kaggle-titanic.git

Cloning into 'kaggle-titanic'...
remote: Enumerating objects: 472, done.
remote: Total 472 (delta 0), reused 0 (delta 0), pack-reused 472
Receiving objects: 100% (472/472), 7.61 MiB | 22.47 MiB/s, done.
Resolving deltas: 100% (216/216), done.


In [0]:
!cd /content


In [10]:
!ls


anscombe.json		      kaggle-titanic	     README.md
california_housing_test.csv   mnist_test.csv
california_housing_train.csv  mnist_train_small.csv


In [11]:
cd /content/


/content


In [12]:
!ls


kaggle-titanic	sample_data


In [13]:
!git clone https://github.com/agconti/kaggle-titanic.git
    

fatal: destination path 'kaggle-titanic' already exists and is not an empty directory.


In [14]:
cd /content/kaggle-titanic/


/content/kaggle-titanic


In [15]:
!ls

 data	  KaggleAux  'Python Examples'	 requirements.txt
 images   LICENSE     ReadMe.md		 Titanic.ipynb


In [0]:
!rm -rf requirements.txt

In [17]:
!ls

 data	  KaggleAux  'Python Examples'	 Titanic.ipynb
 images   LICENSE     ReadMe.md


In [0]:
!cd KaggleAux/

In [19]:
!ls

 data	  KaggleAux  'Python Examples'	 Titanic.ipynb
 images   LICENSE     ReadMe.md


In [0]:
import numpy as np
from pandas import DataFrame
from patsy import dmatrices


def get_dataframe_intersection(df, comparator1, comparator2):
    """
    Return a dataframe with only the columns found in a comparative dataframe.

    Parameters
    ----------
    comparator1: DataFrame
        DataFrame to preform comparison on.
    comparator2: DataFrame
        DataFrame to compare with.

    Returns
    -------
    DataFrame:
        Data frame with columns not found in comparator dropped.

    """
    to_drop = list((c for c in comparator1 if c not in comparator2))
    return df.drop(to_drop, axis=1)


def get_dataframes_intersections(df1, comparator1, df2, comparator2):
    """
    Return DataFrames with the intersection of their column values.

    Parameters
    ----------
    comparator1: DataFrame
        DataFrame to preform comparison on.
    comparator2: DataFrame
        DataFrame to compare with.

    Returns
    -------
    Tuple:
        The resultingDataframe with columns not found in comparator dropped.

    """
    comparator1 = get_dataframe_intersection(df1, comparator1, comparator2)
    comparator2 = get_dataframe_intersection(df2, comparator2, comparator1)
    return comparator1, comparator2


def predict(test_data, results, model_name):
    """
    Return predictions of based on model resutls.

    Parameters
    ----------
    test_data: DataFrame
        should be test data you are trying to predict
    results: dict
        should be dict of your models results wrapper and the formula used
        to produce it.
            ie.
            results['Model_Name'] = {
            [<statsmodels.regression.linear_model.RegressionResultsWrapper> , 
            "Price ~ I(Supply, Demand)]
            }
    model_name: str
        should be the name of your model. You can iterate through the results dict.

    Returns
    -------
    NumPy array
        Predictions in a flat NumPy array.

    Example
    -------
    results = {'Logit': [<statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x117896650>,
               'survived ~ C(pclass) + C(sex) + age + sibsp  + C(embarked)']}
    compared_resuts = predict(test_data, results, 'Logit')

    """
    model_params = DataFrame(results[model_name][0].params)
    formula = results[model_name][1]

    # Create regression friendly test DataFrame
    yt, xt = dmatrices(formula, data=test_data, return_type='dataframe')
    xt, model_params = get_dataframes_intersections(xt, xt.columns,
                                                    model_params, model_params.index)
    # Convert to NumPy arrays for performance
    model_params = np.asarray(model_params)
    yt = np.asarray(yt)
    yt = yt.ravel()
    xt = np.asarray(xt)

    # Use our models to create predictions
    row, col = xt.shape
    model_parameters = model_params.ravel()
    model_array = list((model_parameters for parameter in xrange(row)))
    model_array = np.asarray(model_array)

    # Multiply matrix together
    predictions = np.multiply(xt, model_array)
    predictions = np.sum(predictions, axis=1)
    return predictions


In [22]:
import predict

__all__ = ['predict']

ModuleNotFoundError: ignored